In [1]:
import pandas as pd
import time
import json
from geopy.geocoders import Nominatim
pd.options.mode.copy_on_write = True 

In [2]:
# Lit le fichier de stops
liste_stops_ter = pd.read_csv("./export-ter-gtfs-last/stops.csv", sep=",")
liste_stops_ter = liste_stops_ter[[str.strip(x[:18]) == "StopPoint:OCETrain" for x in liste_stops_ter["stop_id"]]]
liste_stops_ter["location_info"] = ""

In [2]:
geolocator = Nominatim(user_agent="my_geopy_app")
location = geolocator.reverse("48.9523850,7.63409000")
# df = pd.dataframe(location.raw["address"])
print(type(location.raw["address"]))

<class 'dict'>


In [4]:
location.raw["address"]

{'amenity': 'Amazon Hub',
 'road': 'Allée des Tilleuls',
 'village': 'Niederbronn-les-Bains',
 'municipality': 'Haguenau-Wissembourg',
 'county': 'Bas-Rhin',
 'ISO3166-2-lvl6': 'FR-67',
 'state': 'Grand Est',
 'ISO3166-2-lvl4': 'FR-GES',
 'region': 'France métropolitaine',
 'postcode': '67110',
 'country': 'France',
 'country_code': 'fr'}

In [13]:
location_list = { 
    "location_info": [],
    "city": [],
    "stop_id": []
}

count = 1
geolocator = Nominatim(user_agent="my_geopy_app")

for row in liste_stops_ter.itertuples():
    time.sleep(1.5)
    try:
        location = geolocator.reverse(str(row.stop_lat) + "," + str(row.stop_lon))
        data = location.raw["address"]
        location_list["location_info"].append(json.dumps(data))
        if 'town' in data:
            location_list["city"].append(data["town"].upper())
        elif 'village' in data:
            location_list['city'].append(data["village"].upper())
        elif 'city' in data:
            location_list['city'].append(data["city"].upper())
        else:
            location_list['city'].append('Not Found')
        location_list["stop_id"].append(row.stop_id)
    except Exception as e:      # works on python 3.x
        # logger.error('Failed to get infos: %s', repr(e))
        print("An error occured for stop_id ", row.stop_id)
        print("the error is : ", repr(e))
    if count % 100 == 0: 
        pd.DataFrame.from_dict(location_list).to_csv("data_cleaned/stop_location_infos.csv")
    count += 1
df = pd.DataFrame.from_dict(location_list)
df.to_csv("data_cleaned/stop_location_infos.csv")


An error occured for stop_id  StopPoint:OCETrain TER-87353599
the error is :  GeocoderTimedOut('Service timed out')
An error occured for stop_id  StopPoint:OCETrain TER-87413013
the error is :  GeocoderTimedOut('Service timed out')
An error occured for stop_id  StopPoint:OCETrain TER-87473561
the error is :  GeocoderTimedOut('Service timed out')
An error occured for stop_id  StopPoint:OCETrain TER-87722207
the error is :  GeocoderTimedOut('Service timed out')
An error occured for stop_id  StopPoint:OCETrain TER-87751719
the error is :  GeocoderTimedOut('Service timed out')


In [10]:
liste_city = pd.read_csv("./data_cleaned/stop_location_infos_archives-1.csv", sep=",")

location_list = { 
    "location_info": [],
    "city": [],
    "stop_id": []
}

for row in liste_city.itertuples():
    try:
        location_list["location_info"].append(row.location_info)
        data = json.loads(row.location_info)
        if 'town' in data:
            location_list["city"].append(data["town"].upper())
        elif 'village' in data:
            location_list['city'].append(data["village"].upper())
        elif 'city' in data:
            location_list['city'].append(data["city"].upper())
        else:
            location_list['city'].append('Not Found')
        location_list["stop_id"].append(row.stop_id)
    except Exception as e:      # works on python 3.x
        # logger.error('Failed to get infos: %s', repr(e))
        print("An error occured for stop_id ", row.stop_id)
        print("the error is : ", repr(e))
df = pd.DataFrame.from_dict(location_list)
df.to_csv("data_cleaned/stop_location_infos.csv")

{'location_info': ['{"railway": "Portbou", "road": "Carrer Claudi Planas", "village": "Portbou", "county": "Alt Empord\\u00e0", "province": "Girona", "ISO3166-2-lvl6": "ES-GI", "state": "Catalunya", "ISO3166-2-lvl4": "ES-CT", "postcode": "17497", "country": "Espa\\u00f1a", "country_code": "es"}', '{"railway": "Appenweier", "road": "Bahnhofstra\\u00dfe", "village": "Appenweier", "county": "Ortenaukreis", "state": "Baden-W\\u00fcrttemberg", "ISO3166-2-lvl4": "DE-BW", "postcode": "77767", "country": "Deutschland", "country_code": "de"}', '{"house_number": "69", "road": "Bahnhofstra\\u00dfe", "village": "Legelshurst", "county": "Ortenaukreis", "state": "Baden-W\\u00fcrttemberg", "ISO3166-2-lvl4": "DE-BW", "postcode": "77731", "country": "Deutschland", "country_code": "de"}', '{"house_number": "24", "road": "Gerbereistra\\u00dfe", "village": "Kork", "town": "Kehl", "county": "Ortenaukreis", "state": "Baden-W\\u00fcrttemberg", "ISO3166-2-lvl4": "DE-BW", "postcode": "77694", "country": "Deuts